### Data Prepocessing Step

In [1]:
import pandas as pd

In [2]:
import os

In [3]:
%pwd

'd:\\Silent_Night\\mlops\\AML_Classification\\research'

In [4]:
os.chdir("../")

In [78]:
%pwd

'd:\\Silent_Night\\mlops\\AML_Classification'

In [29]:
df=pd.read_csv("artifacts/data_ingestion/data.csv")
categorical_feature = ['receiving_currency', 'Payment_Currency', 'Payment Format',"is_laundering"]
continuous_feature = ['from_bank', 'to_bank', 'amount_received', 'amount_paid']

In [30]:
from sklearn.preprocessing import StandardScaler,LabelEncoder
le=LabelEncoder()
sc=StandardScaler()

In [31]:
df1=pd.DataFrame
sc.fit(df[continuous_feature])

StandardScaler()

In [38]:
df1=sc.transform(df[continuous_feature])
df1 = pd.DataFrame(df1, columns=continuous_feature)

In [39]:
print(df1)

      from_bank   to_bank  amount_received  amount_paid
0     -0.590439 -0.481927        -0.018969    -0.018856
1     -0.590439  1.854354        -0.019002    -0.018889
2     -0.590439 -0.621317        -0.018044    -0.017931
3     -0.438094  2.300571        -0.019032    -0.018919
4     -0.590439 -0.613179        -0.019031    -0.018918
...         ...       ...              ...          ...
9833  -0.363817 -0.541974        -0.019031    -0.018918
9834  -0.591208 -0.725184        -0.018211    -0.018098
9835  -0.428273 -0.613964        -0.019031    -0.018918
9836  -0.191446 -0.376734        -0.019032    -0.018919
9837  -0.549568 -0.740589        -0.019033    -0.018919

[9838 rows x 4 columns]


In [50]:
le = LabelEncoder()
le.fit(df['receiving_currency'])
df_le=le.transform(df['receiving_currency'])
df_le1=pd.DataFrame(df_le,columns=['receiving_currency'])
le.__dict__

{'classes_': array(['Australian Dollar', 'Bitcoin', 'Brazil Real', 'Canadian Dollar',
        'Euro', 'Mexican Peso', 'Ruble', 'Rupee', 'Saudi Riyal', 'Shekel',
        'Swiss Franc', 'UK Pound', 'US Dollar', 'Yen', 'Yuan'],
       dtype=object)}

In [52]:
import pickle

In [53]:
df_le=pd.DataFrame()
for i in categorical_feature:
    le.fit_transform(df[[i]])
    with open(f"artifacts/model_data/le_{i}.plk","wb") as file:
        pickle.dump(le,file)

c:\Users\JOHN\anaconda3\envs\AML\lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\JOHN\anaconda3\envs\AML\lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\JOHN\anaconda3\envs\AML\lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\JOHN\anaconda3\envs\AML\lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected.

In [51]:
print("Label Mapping:", dict(zip(le.classes_, range(len(le.classes_)))))

Label Mapping: {'Australian Dollar': 0, 'Bitcoin': 1, 'Brazil Real': 2, 'Canadian Dollar': 3, 'Euro': 4, 'Mexican Peso': 5, 'Ruble': 6, 'Rupee': 7, 'Saudi Riyal': 8, 'Shekel': 9, 'Swiss Franc': 10, 'UK Pound': 11, 'US Dollar': 12, 'Yen': 13, 'Yuan': 14}


In [18]:
#update entity
from dataclasses import dataclass 
from pathlib import Path

@dataclass(frozen=True)# To ensure that we give correct type of input as specified for the data ingestion process.
class dataprocessingconfig:
    root_dir: Path
    drop_feature: list
    categorical_feature: list
    continuous_feature: list
    save_processed_data: Path
    save_label_encoder: Path
    save_scaler_obj: Path
    downloaded_data: Path

In [19]:
#update configration manger
from AML_Classifier.constants.__init__ import CONFIG_FILE_PATH,PARAMS_FILE_PATH
from AML_Classifier.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self,config_filepath = CONFIG_FILE_PATH,params_filepath = PARAMS_FILE_PATH):
        self.config= read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_process_config(self) -> dataprocessingconfig:
        config = self.config.data_prepocess
        create_directories([config.root_dir])
        downloaded_data = os.path.join(self.config.data_ingestion.root_dir,"data.csv")

        data_process_config= dataprocessingconfig(
            root_dir = Path(config.root_dir),
            drop_feature=config.drop_feature,
            categorical_feature = config.categorical_feature,
            continuous_feature = config.continuous_feature,
            downloaded_data=Path(downloaded_data),
            save_processed_data=Path(config.save_processed_data),
            save_label_encoder  = Path(config.save_label_encoder),
            save_scaler_obj = Path(config.save_scaler_obj)
            )
        return data_process_config

In [79]:
#update the compontents
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
import pickle
import numpy as np

In [92]:
class Processdata:
    def __init__(self,config: dataprocessingconfig):
        self.config = config

    def preprocessing(self):
        df=pd.read_csv(self.config.downloaded_data)
        #Drop the unnessary feature
        df=df.drop(self.config.drop_feature,axis=1)
        #Intialize a dataframe
        processed_df=pd.DataFrame()

        #Label encoding
        df_le=pd.DataFrame()
        le = LabelEncoder()
        for i in self.config.categorical_feature:
            num_df=np.array(df[[i]])
            df_le[i]=le.fit_transform(num_df.ravel())
            with open(f"{self.config.save_label_encoder}/le_{i}.pkl","wb") as file:
                pickle.dump(le,file)

        #Scaling the continous variable
        sc = StandardScaler()
        sc.fit(df[self.config.continuous_feature])
        df_sc=sc.transform(df[self.config.continuous_feature])
        df_sc1=pd.DataFrame(df_sc, columns=self.config.continuous_feature)

        processed_df=pd.concat([df_le,df_sc1], axis=1)

        #Save the Scaler
        with open(self.config.save_scaler_obj,"wb") as file:
            pickle.dump(sc, file)
        

        #Save the file
        processed_df.to_csv(self.config.save_processed_data,index=False)

In [94]:
#pipeline
try:
    config = ConfigurationManager()
    get_data_process_config = config.get_data_process_config()
    Process_data = Processdata(config = get_data_process_config)
    Process_data.preprocessing()
except Exception as e:
    raise e

[2024-07-03 14:35:03,895: INFO: common: yaml file: config\config.yaml loaded successfully]


[2024-07-03 14:35:03,895: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-03 14:35:03,903: INFO: common: created directory at: artifacts]
[2024-07-03 14:35:03,903: INFO: common: created directory at: artifacts/model_data]
